In [1]:
import pandas as pd
from ConectionSQL import SQL

In [12]:
sql=SQL('Netfeira','sqlserver','MOINHO','192.168.0.252')

conectando=sql.ConexaoSQL()

pd.set_option('float_format','{:.2f}'.format)

In [13]:
querys={
    
    'Faltas':"""
    
    WITH TabTpCorte (cd_tp_faltaprd,descrica) AS (

    SELECT cd_tp_faltaprd,descricao
    FROM tp_faltaprd
    WHERE ativo=1
    --AND descricao LIKE '%CORTE%'
    ),

    TabFalta (dt_falta,cd_vend,cd_clien,nu_ped,cd_prod,unid_vda,qtde_falta_vda,preco_unit,vl_total) AS (

    SELECT CONVERT(DATETIME,CAST(dt_falta AS date),101) AS dt_falta,
    cd_vend,cd_clien,nu_ped,cd_prod,unid_vda,qtde_falta,preco_unit,
    ROUND(qtde_falta*preco_basico,2) AS vl_total
    FROM faltaprd
    INNER JOIN TabTpCorte ON faltaprd.cd_tp_faltaprd=TabTpCorte.cd_tp_faltaprd
    )

    SELECT dt_falta AS 'Data de Falta',
    TabFalta.cd_vend AS 'ID Vendedor',
    TabFalta.cd_clien AS 'ID Cliente',cliente.nome_res AS 'Nome Fantasia',
    nu_ped AS 'Pedido',TabFalta.cd_prod AS 'SKU',produto.descricao AS 'Produto',
    unid_vda AS 'Unid. VDA',qtde_falta_vda AS 'Qtde. VDA',
    preco_unit AS 'Valor Unitário',vl_total AS 'Total do Pedido'
    FROM TabFalta
    --INNER JOIN vendedor ON TabFalta.cd_vend=vendedor.cd_vend
    INNER JOIN cliente ON TabFalta.cd_clien=cliente.cd_clien
    INNER JOIN produto ON TabFalta.cd_prod=produto.cd_prod
    WHERE YEAR(dt_falta)=YEAR(GETDATE()) AND MONTH(dt_falta)=MONTH(GETDATE())
    AND DAY(dt_falta)=DAY(GETDate())
    
    """,
    
    'Vendedores':"""
    
    SELECT V.CD_VEND AS 'ID Vendedor',V.NOME AS 'Vendedor',
    CASE WHEN 
    CHARINDEX(' ',LTRIM(RTRIM(V.NOME)))=0 THEN V.NOME ELSE
    LTRIM(RTRIM(LEFT(V.NOME,CHARINDEX(' ',LTRIM(RTRIM(V.NOME))))))+ ' '+
    LTRIM(RTRIM(RIGHT(V.NOME,CHARINDEX(' ',REVERSE(LTRIM(RTRIM(V.NOME))))))) 
    END AS 'Nome Resumido',V.CD_EQUIPE AS 'ID Equipe',
    U.E_MAIL AS 'E-mail',categ.descricao AS 'Categoria'
    FROM VENDEDOR AS V
    LEFT JOIN USUARIO AS U ON V.CD_VEND=U.CD_USUARIO
    LEFT JOIN categ ON V.CATEG=categ.categ
    WHERE V.ATIVO=1    
    
    
    """,
    
    'Supervisor':"""
    
    WITH TabVendedor(cd_vend,nome,nome_resumido,st_vend,cd_equipe,e_mail,categoria) AS (

    SELECT cd_vend,vendedor.nome,
    CASE WHEN CHARINDEX(' ',LTRIM(RTRIM(vendedor.nome)))>0 THEN
    LTRIM(RTRIM(LEFT(vendedor.nome,CHARINDEX(' ',LTRIM(RTRIM(vendedor.nome)))))) + ' '+
    LTRIM(RTRIM(RIGHT(vendedor.nome,CHARINDEX(' ',REVERSE(LTRIM(RTRIM(vendedor.nome)))))))
    ELSE vendedor.nome END AS nome_resumido,
    CASE WHEN vendedor.ativo=1 THEN 'ATIVO' ELSE 'INATIVO' END AS st_vend,
    cd_equipe,usuario.e_mail,categ.descricao AS	categoria
    FROM vendedor
    LEFT JOIN usuario ON vendedor.cd_vend=usuario.cd_usuario
    LEFT JOIN categ ON vendedor.categ=categ.categ),

    TabSupervisor (cd_equipe,equipe,cd_vend_sup,supervisor,email_supervisor,cd_gerente,gerente,email_gerente) AS (

    SELECT supervisor.cd_equipe,supervisor.equipe,cd_vend_sup,
    supervisor,email_supervisor,
    supervisor.cd_gerente,TabVendedor.nome AS gerente,
    TabVendedor.e_mail AS email_gerente
    FROM (
    SELECT equipe.cd_equipe,equipe.descricao AS equipe,cd_vend_sup,
    TabVendedor.nome_resumido AS supervisor,e_mail AS email_supervisor,
    gerencia.descricao AS cd_gerente
    FROM equipe
    INNER JOIN TabVendedor ON equipe.cd_vend_sup=TabVendedor.cd_vend
    INNER JOIN gerencia ON equipe.cd_gerencia=gerencia.cd_gerencia
    )supervisor
    INNER JOIN TabVendedor ON supervisor.cd_gerente=TabVendedor.cd_vend)

    SELECT cd_equipe AS 'ID Equipe',equipe AS 'Equipe',cd_vend_sup AS 'ID Sup',
    supervisor AS 'Supervisor',email_supervisor AS 'Email Sup',
    cd_gerente AS 'ID Gerente',gerente AS 'Gerente',email_gerente AS 'Email Gerente'
    FROM TabSupervisor    
    
    """
    
}

In [14]:
tabela_dict=dict()

for tabela,query in querys.items():
    
    tabela_dict[tabela]=pd.read_sql(query,conectando)
    
    pass

In [15]:
tabela_dict['Faltas'].columns

Index(['Data de Falta', 'ID Vendedor', 'ID Cliente', 'Nome Fantasia', 'Pedido',
       'SKU', 'Produto', 'Unid. VDA', 'Qtde. VDA', 'Valor Unitário',
       'Total do Pedido'],
      dtype='object')

In [16]:
tabela_dict['Vendedores'].columns

Index(['ID Vendedor', 'Vendedor', 'Nome Resumido', 'ID Equipe', 'E-mail',
       'Categoria'],
      dtype='object')

In [17]:
tabela_dict['Supervisor'].columns

Index(['ID Equipe', 'Equipe', 'ID Sup', 'Supervisor', 'Email Sup',
       'ID Gerente', 'Gerente', 'Email Gerente'],
      dtype='object')

In [21]:
tabela_dict['Faltas']=tabela_dict['Faltas'].merge(tabela_dict['Vendedores'],on='ID Vendedor',how='inner')[['Data de Falta', 'ID Vendedor','Vendedor', 'Nome Resumido', 'ID Equipe', 'E-mail',
       'Categoria', 'ID Cliente', 'Nome Fantasia', 'Pedido',
       'SKU', 'Produto', 'Unid. VDA', 'Qtde. VDA', 'Valor Unitário',
       'Total do Pedido']]

In [27]:
tabela_dict['Faltas']=tabela_dict['Faltas'].merge(tabela_dict['Supervisor'],on='ID Equipe',how='inner')[['Data de Falta', 'ID Vendedor','Vendedor', 'Nome Resumido', 'Equipe', 'E-mail',
       'Categoria','Supervisor', 'Email Sup',
       'Gerente', 'Email Gerente', 'ID Cliente', 'Nome Fantasia', 'Pedido',
       'SKU', 'Produto', 'Unid. VDA', 'Qtde. VDA', 'Valor Unitário',
       'Total do Pedido']]

In [30]:
tabela_dict['Faltas'].sort_values('Total do Pedido',ascending=False,inplace=True)

tabela_dict['Faltas']

,Data de Falta,ID Vendedor,Vendedor,Nome Resumido,Equipe,E-mail,Categoria,Supervisor,Email Sup,Gerente,Email Gerente,ID Cliente,Nome Fantasia,Pedido,SKU,Produto,Unid. VDA,Qtde. VDA,Valor Unitário,Total do Pedido
2,2022-06-02,RAQUELSO,RAQUEL SOBRINHO,RAQUEL SOBRINHO,EQUIPE 9 (ATIVO),RAQUEL@DEMARCHISAOPAULO.COM.BR,CLT,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,27934,MERCEARIA DUCARMO,402993,850,"CONGELADO BATATA MCCAIN RUSTICA WEDGES (2,5KG)",UN,2.00,52.86,105.72
1,2022-06-02,RENATAAP,RENATA APARECIDA NOCHIERI COSTA,RENATA COSTA,EQUIPE 9 (ATIVO),RENATA@DEMARCHISAOPAULO.COM.BR,CLT,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,19833,DAS BROT CIDADE JD,402899,9385,"POLPA DE MORANGO (1,2KG) FOOD",UN,5.00,20.98,104.90
6,2022-06-02,LEOLOREN,LEONARDO LORENCETTI,LEONARDO LORENCETTI,EQUIPE 3 (CLT),LEO.LORENCETTI@GMAIL.COM,CLT,ULISSES BACCI,ULISSES@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,16476,CHAMBERY,402961,1002,"CONGELADO MIX DE LEGUMES (1,2KG )",UN,6.00,14.25,99.06
3,2022-06-02,ROBSONAN,ROBSON DOMINGUES DE ANDRADE,ROBSON ANDRADE,EQUIPE 4 (VAR),ROBSON_DOMINGUES1@HOTMAI.COM,CLT,DOUGLAS TEDESCHI,DOUGLASMARTINS@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,9277,CHAMA SUPERMERCADOS,403003,9539,"VERO SORBETTO MANGA 1,2KG/1,5L",UN,6.00,19.78,60.84
4,2022-06-02,ROBSONAN,ROBSON DOMINGUES DE ANDRADE,ROBSON ANDRADE,EQUIPE 4 (VAR),ROBSON_DOMINGUES1@HOTMAI.COM,CLT,DOUGLAS TEDESCHI,DOUGLASMARTINS@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,9274,CHAMA SUPERMERCADOS,402992,9539,"VERO SORBETTO MANGA 1,2KG/1,5L",UN,6.00,19.78,60.84
5,2022-06-02,LEOLOREN,LEONARDO LORENCETTI,LEONARDO LORENCETTI,EQUIPE 3 (CLT),LEO.LORENCETTI@GMAIL.COM,CLT,ULISSES BACCI,ULISSES@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,4897,PAULISTANIA PANIFICA,403013,1072,CONGELADO RISOLE QUEIJO ORIGINAL 6X1000G,UN,1.00,55.76,55.76
0,2022-06-02,RENATAAP,RENATA APARECIDA NOCHIERI COSTA,RENATA COSTA,EQUIPE 9 (ATIVO),RENATA@DEMARCHISAOPAULO.COM.BR,CLT,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,25746,CARLA MARGIT,402907,9340,CONGELADO PAO 5 CEREAIS PRE ASSADO 220GR,UN,1.00,11.66,11.66


In [34]:
dados_df=tabela_dict['Faltas']

In [56]:
emails={'E-mail':dados_df['E-mail'].loc[(dados_df['E-mail']!='')&(dados_df['Categoria']=='CLT')].unique().tolist(),
        
        'Email Sup': dados_df['Email Sup'].loc[(dados_df['Email Sup']!='')].unique().tolist()
        
       }

colunas={'E-mail':'Email Sup','Email Sup':'Email Gerente'}

for coluna,email in emails.items():
    
    for env in email:
        
        email_to=[]
        
        email_cc=[]
        
        email_res=dados_df[colunas[coluna]].loc[dados_df[coluna]==env].unique().tolist()
        
        if(coluna=='E-mail'):
        
            nome=dados_df['Nome Resumido'].loc[dados_df[coluna]==env].unique().tolist()
            
            nome=str(nome[0]).title()
            
            arq=(f'{nome}.xlsx')
            
            dados_df.loc[dados_df[coluna]==env].to_excel(arq,index=False,encoding='ISO-8859-1')
            
            pass
        
        else:
            
            nome=dados_df['Supervisor'].loc[dados_df[coluna]==env].unique().tolist()
            
            nome=str(nome[0]).title()
            
            arq=(f'{nome}.xlsx')
            
            dados_df.loc[dados_df[coluna]==env].to_excel(arq,index=False,encoding='ISO-8859-1')
            
            pass
        
        pass
    
    pass

In [52]:
produto_df=dados_df[['SKU','Produto','Total do Pedido']].groupby(['SKU','Produto'],as_index=False).agg({'Total do Pedido':'sum'})

produto_df.sort_values('Total do Pedido',ascending=False,inplace=True)

In [61]:
pedido=len(dados_df['Pedido'].unique().tolist())

sku=len(dados_df['SKU'].unique().tolist())

total=round(dados_df['Total do Pedido'].sum(),2)
          
round(total,2)

498.78